# Tutorial 3: Clipper in Action with Pong

We have already explored some of the features of Clipper (insert recap here). Now let's take a look at Clipper in action with Pong! Released by Atari in 1972, Pong was the first commercially successful video game. You can read more about it <a href="https://en.wikipedia.org/wiki/Pong">here</a>

The goal of this tutorial is to use Clipper to deploy several AI policies to play against, and in doing so explore:
(list of Clipper features we want to highlight)

This tutorial will be broken up into 3 main parts:
#### 1. Starting Clipper and Deploying First Policy
#### 2. Training a Better Model
#### 3. Deploying an updated, trained model

## Part 1: Starting Clipper and Deploying First Policy

Install packaged env from requirements.txt (not sure if we should do this before opening jupyter notebook or not)

In [1]:
#Import dependencies
from clipper_admin import ClipperConnection, DockerContainerManager
from clipper_admin.deployers import python as py_deployer
import random
import numpy as np
import pandas as pd
from datascience import *
from sklearn import linear_model
from IPython.display import Image
from IPython.core.display import HTML 


Our first step is to get Clipper started and deploy our first AI policy: one that guesses which direction to move the paddle randomly. First, start Docker and the node server in your bash.

The cell below will start Clipper. You can run docker ps in your bash to see the Clipper Docker containers

In [2]:
#Starting Clipper, make sure you have Docker running before you run this cell
clipper_conn = ClipperConnection(DockerContainerManager())
clipper_conn.stop_all()
clipper_conn.start_clipper()

18-09-06:11:25:51 INFO     [clipper_admin.py:1258] Stopped all Clipper cluster and all model containers
18-09-06:11:25:51 INFO     [docker_container_manager.py:119] Starting managed Redis instance in Docker
18-09-06:11:25:54 INFO     [clipper_admin.py:126] Clipper is running


Running the cell below will register an application in Clipper called "pong" and create a Clipper endpoint for our random policy at http://localhost:1337/pong/predict

In [3]:
def random_predict(xs):
    action = random.randint(0, 2)
    return [str(action) for _ in xs]

py_deployer.create_endpoint(clipper_conn, name="pong", input_type="doubles", func=random_predict,
                            default_output="0", slo_micros=100000)

18-09-06:11:25:57 INFO     [clipper_admin.py:201] Application pong was successfully registered
18-09-06:11:25:57 INFO     [deployer_utils.py:44] Saving function to /tmp/clipper/tmp_9u9zzpl
18-09-06:11:25:57 INFO     [deployer_utils.py:54] Serialized and supplied predict function
18-09-06:11:25:57 INFO     [python.py:192] Python closure saved
18-09-06:11:25:57 INFO     [python.py:202] Using Python 3.5 base image
18-09-06:11:25:57 INFO     [clipper_admin.py:452] Building model Docker image with model data from /tmp/clipper/tmp_9u9zzpl
18-09-06:11:25:57 INFO     [clipper_admin.py:456] {'stream': 'Step 1/2 : FROM clipper/python35-closure-container:0.3.0'}
18-09-06:11:25:57 INFO     [clipper_admin.py:456] {'stream': '\n'}
18-09-06:11:25:57 INFO     [clipper_admin.py:456] {'stream': ' ---> 1fc910c1ed28\n'}
18-09-06:11:25:57 INFO     [clipper_admin.py:456] {'stream': 'Step 2/2 : COPY /tmp/clipper/tmp_9u9zzpl /model/'}
18-09-06:11:25:57 INFO     [clipper_admin.py:456] {'stream': '\n'}
18-09-06

Now that we have a model deployed, let's see how it works! Run the cell below to start the game

In [4]:
from subprocess import Popen

server_proc = Popen(["python", "pong-server.py", "localhost:1337", "pong_server.log"])

18-09-06:11:26:09 INFO     [pong-server.py:109] Starting Pong Server on localhost:3000
18-09-06:11:26:10 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/clipper-tutorials/static/index.html
127.0.0.1 - - [06/Sep/2018 11:26:10] "GET /pong/ HTTP/1.1" 200 -
18-09-06:11:26:10 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/clipper-tutorials/static/game.js
127.0.0.1 - - [06/Sep/2018 11:26:10] "GET /pong/game.js HTTP/1.1" 200 -
18-09-06:11:26:10 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/clipper-tutorials/static/pong.css
127.0.0.1 - - [06/Sep/2018 11:26:10] "GET /pong/pong.css HTTP/1.1" 200 -
18-09-06:11:26:10 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/clipper-tutorials/static/pong.js
127.0.0.1 - - [06/Sep/2018 11:26:10] "GET /pong/pong.js HTTP/1.1" 200 -
18-09-06:11:26:10 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/clipper-tutorials/static/images/press1.png
18-09-06:11:

<_io.BufferedReader name=9>
18-09-06:11:26:14 INFO     [pong-server.py:73] 142
142
18-09-06:11:26:14 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.42039599999999994, 0.185022432, 0.7825994240000002, 0.32376799999999994, -0.32376799999999994, 0.16885403199999996, 0.7987278240000002]}
127.0.0.1 - - [06/Sep/2018 11:26:14] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":11,"output":2,"default":false}' in 12.1 ms
{'Content-Type': 'application/json', 'Content-Length': '42'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=9>
18-09-06:11:26:14 INFO     [pong-server.py:73] 134
134
18-09-06:11:26:14 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.42039599999999994, 0.201230832, 0.7664310240000001, 0.32456799999999997, -0.32456799999999997, 0.185022432, 0.7825994240000002]}
127.0.0.1 - - [06/Sep/2018 11:26:14] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":12,"ou

<_io.BufferedReader name=9>
18-09-06:11:26:15 INFO     [pong-server.py:73] 145
145
18-09-06:11:26:15 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.3209999999999999, 0.4491568319999999, 0.5192763200000002, 0.3365679999999999, -0.3365679999999999, 0.4326842079999999, 0.5357105280000002]}
127.0.0.1 - - [06/Sep/2018 11:26:15] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":26,"output":1,"default":false}' in 16.965 ms
{'Content-Type': 'application/json', 'Content-Length': '42'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=9>
18-09-06:11:26:15 INFO     [pong-server.py:73] 146
146
18-09-06:11:26:15 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.30080399999999985, 0.4663426079999999, 0.5021321600000003, 0.3373839999999999, -0.3373839999999999, 0.4491568319999999, 0.5192763200000002]}
127.0.0.1 - - [06/Sep/2018 11:26:15] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{

Clipper responded with '{"query_id":40,"output":1,"default":false}' in 18.103 ms
{'Content-Type': 'application/json', 'Content-Length': '42'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=9>
18-09-06:11:26:15 INFO     [pong-server.py:73] 150
150
18-09-06:11:26:15 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.28337999999999985, 0.7064314079999997, 0.26260387200000035, 0.34858399999999984, -0.34858399999999984, 0.6886744319999997, 0.28031923200000036]}
127.0.0.1 - - [06/Sep/2018 11:26:15] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":41,"output":2,"default":false}' in 14.808 ms
{'Content-Type': 'application/json', 'Content-Length': '42'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=9>
18-09-06:11:26:15 INFO     [pong-server.py:73] 148
148
18-09-06:11:26:15 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.3031799999999999, 0.7238806079999998, 0.24

127.0.0.1 - - [06/Sep/2018 11:26:16] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":55,"output":0,"default":false}' in 26.897000000000002 ms
{'Content-Type': 'application/json', 'Content-Length': '42'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=9>
18-09-06:11:26:16 INFO     [pong-server.py:73] 147
147
18-09-06:11:26:16 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.37841999999999987, 0.9723694079999996, 0.06818327999999999, 0.3605839999999998, 0.3605839999999998, 0.9540004319999996, 0.04981430399999998]}
127.0.0.1 - - [06/Sep/2018 11:26:16] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":56,"output":2,"default":false}' in 20.802999999999997 ms
{'Content-Type': 'application/json', 'Content-Length': '42'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=9>
18-09-06:11:26:16 INFO     [pong-server.py:73] 149
149
18-09-06:11:26:16 INFO     [pong-server.py:

<_io.BufferedReader name=9>
18-09-06:11:26:17 INFO     [pong-server.py:73] 143
143
18-09-06:11:26:17 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.3772319999999998, 1.228326431999999, 0.3241403039999996, 0.3717679999999997, 0.3717679999999997, 1.210129007999999, 0.30594287999999964]}
127.0.0.1 - - [06/Sep/2018 11:26:17] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":70,"output":0,"default":false}' in 20.386000000000003 ms
{'Content-Type': 'application/json', 'Content-Length': '42'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=9>
18-09-06:11:26:17 INFO     [pong-server.py:73] 133
133
18-09-06:11:26:17 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.3574319999999998, 1.246, 0.34274870399999957, -0.3725679999999997, 0.18628399999999984, 1.228326431999999, 0.3241403039999996]}
127.0.0.1 - - [06/Sep/2018 11:26:17] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"qu

<_io.BufferedReader name=9>
18-09-06:11:26:17 INFO     [pong-server.py:73] 148
148
18-09-06:11:26:17 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.4239599999999998, 0.9814615840000002, 0.47726499599999944, -0.3837839999999996, 0.19749999999999993, 1.0005907840000001, 0.46740999599999944]}
127.0.0.1 - - [06/Sep/2018 11:26:17] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":85,"output":0,"default":false}' in 25.642000000000003 ms
{'Content-Type': 'application/json', 'Content-Length': '42'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=9>
18-09-06:11:26:18 INFO     [pong-server.py:73] 148
148
18-09-06:11:26:18 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.4045559999999998, 0.9626753760000002, 0.48696170399999944, -0.38456799999999963, 0.1982839999999999, 0.9814615840000002, 0.47726499599999944]}
127.0.0.1 - - [06/Sep/2018 11:26:18] "POST /pong/predict HTTP/1.1" 200 -
Clipper 

18-09-06:11:26:18 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/clipper-tutorials/static/sounds/goal.wav
127.0.0.1 - - [06/Sep/2018 11:26:18] "GET /pong/sounds/ping.wav HTTP/1.1" 200 -
127.0.0.1 - - [06/Sep/2018 11:26:18] "GET /pong/sounds/pong.wav HTTP/1.1" 200 -
127.0.0.1 - - [06/Sep/2018 11:26:18] "GET /pong/sounds/wall.wav HTTP/1.1" 200 -
127.0.0.1 - - [06/Sep/2018 11:26:18] "GET /pong/sounds/goal.wav HTTP/1.1" 200 -
18-09-06:11:26:18 INFO     [pong-server.py:51] Local path: /favicon.ico
127.0.0.1 - - [06/Sep/2018 11:26:18] code 403, message Forbidden
127.0.0.1 - - [06/Sep/2018 11:26:18] "GET /favicon.ico HTTP/1.1" 403 -
18-09-06:11:27:01 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/clipper-tutorials/static/index.html
127.0.0.1 - - [06/Sep/2018 11:27:01] "GET /pong/ HTTP/1.1" 200 -
18-09-06:11:27:01 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/clipper-tutorials/static/game.js
127.0.0.1 - - [06/Sep/2018 11:27:

Congratulations! We have depolyed our first model to Clipper! Let's take a look at it by clicking http://localhost:3000/pong/ and pressing 1 to start the game.

## Part 2: Training a better model

As you probably noticed, the random-guessing policy did not perform well at all. In order to train a better model, we are going to use imitation learning. Let's play the game again, this time with 2 actual players, and we will collect data on how you play the game to train our model! 

Go to http://localhost:3000/pong/ and press 2. Find someone next to you and play pong against them!!

<b>Part to collect data into a output.csv --> currently doing this in really hacky way to just get it to work, but trying to figure out better solution</b>

Now that we have our data, lets train a new model! First, run the cell below to clean the data and format it for scikit-learn's LogisticRegression model.

In [20]:
df_data = pd.read_csv('out.csv')

def convert_label(label):
    """Convert labels into numeric values"""
    if(label=="down"):
        return 1
    elif(label=="up"):
        return 2
    else:
        return 0

df_data['label'] = df['label'].apply(convert_label)
df_data.loc[:, "leftPaddle_y":"ball_y_prev"] = df.loc[:, "leftPaddle_y":"ball_y_prev"]/500.0

df_data.head()

,label,leftPaddle_y,ball_x,ball_y,ball_dx,ball_dy,ball_x_prev,ball_y_prev
0,1,0.420000,0.153370,0.189479,0.322200,0.322200,0.137280,0.173389
1,1,0.636612,0.332007,0.368116,0.330952,0.330952,0.316140,0.352249
2,1,0.676608,0.365515,0.401624,0.332568,0.332568,0.348906,0.385016
3,1,0.717396,0.399854,0.435963,0.334216,0.334216,0.385498,0.421607
4,1,0.736800,0.416250,0.452359,0.335000,0.335000,0.399854,0.435963


We are going to use the data to train a scikit-learn Logistic Regression model. You can read more about the particular model <a url="http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html">here</a>

In [6]:
labels = data_df['label']
training_data= data_df.drop(['label'], axis=1)

model = linear_model.LogisticRegression()
model.fit(training_data, labels)



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## Part 3: Deploying updated model

Now that we have an updated model, we can deploy that model onto Clipper. Once the system realizes there is a new version of the model, it will automatically switch to serving the newer version of the model.

![update_model](update_model.png)

In [7]:
def predict(inputs):
    # model.predict returns a list of predictions
    preds = model.predict(inputs)
    return [str(p) for p in preds]

py_deployer.deploy_python_closure(clipper_conn, name="pong", version=3, input_type="doubles", func=predict, pkgs_to_install=["numpy","scipy", "pandas", "datascience", "sklearn"])


18-09-06:11:28:47 INFO     [deployer_utils.py:44] Saving function to /tmp/clipper/tmpyf4jimka
18-09-06:11:28:47 INFO     [deployer_utils.py:54] Serialized and supplied predict function
18-09-06:11:28:47 INFO     [python.py:192] Python closure saved
18-09-06:11:28:47 INFO     [python.py:202] Using Python 3.5 base image
18-09-06:11:28:47 INFO     [clipper_admin.py:452] Building model Docker image with model data from /tmp/clipper/tmpyf4jimka
18-09-06:11:29:35 INFO     [clipper_admin.py:456] {'stream': 'Step 1/3 : FROM clipper/python35-closure-container:0.3.0'}
18-09-06:11:29:35 INFO     [clipper_admin.py:456] {'stream': '\n'}
18-09-06:11:29:35 INFO     [clipper_admin.py:456] {'stream': ' ---> 1fc910c1ed28\n'}
18-09-06:11:29:35 INFO     [clipper_admin.py:456] {'stream': 'Step 2/3 : COPY /tmp/clipper/tmpyf4jimka /model/'}
18-09-06:11:29:35 INFO     [clipper_admin.py:456] {'stream': '\n'}
18-09-06:11:29:35 INFO     [clipper_admin.py:456] {'stream': ' ---> 85c2e7a1062c\n'}
18-09-06:11:29:35 

18-09-06:11:29:35 INFO     [clipper_admin.py:456] {'stream': '  Downloading https://files.pythonhosted.org/packages/ad/c2/b500ea05d5f9f361a562f089fc91f77ed3b4783e13a08a3daf82069b1224/packaging-17.1-py2.py3-none-any.whl\n'}
18-09-06:11:29:35 INFO     [clipper_admin.py:456] {'stream': 'Collecting imagesize (from sphinx->datascience)\n'}
18-09-06:11:29:35 INFO     [clipper_admin.py:456] {'stream': '  Downloading https://files.pythonhosted.org/packages/fc/b6/aef66b4c52a6ad6ac18cf6ebc5731ed06d8c9ae4d3b2d9951f261150be67/imagesize-1.1.0-py2.py3-none-any.whl\n'}
18-09-06:11:29:35 INFO     [clipper_admin.py:456] {'stream': 'Requirement already satisfied: requests>=2.0.0 in /usr/local/lib/python3.5/site-packages (from sphinx->datascience)\n'}
18-09-06:11:29:35 INFO     [clipper_admin.py:456] {'stream': 'Collecting Pygments>=2.0 (from sphinx->datascience)\n'}
18-09-06:11:29:35 INFO     [clipper_admin.py:456] {'stream': '  Downloading https://files.pythonhosted.org/packages/02/ee/b6e02dc6529e82b75

18-09-06:11:29:35 INFO     [clipper_admin.py:456] {'stream': 'Successfully installed Pygments-2.2.0 alabaster-0.7.11 atomicwrites-1.2.1 attrs-18.2.0 babel-2.6.0 coverage-3.7.1 coveralls-0.5 datascience-0.10.5 docopt-0.6.2 docutils-0.14 folium-0.2.1 imagesize-1.1.0 more-itertools-4.3.0 packaging-17.1 pandas-0.23.4 pathlib2-2.3.2 pluggy-0.7.1 py-1.6.0 pyparsing-2.2.0 pytest-3.8.0 pytz-2018.5 scikit-learn-0.19.2 scipy-1.1.0 sklearn-0.0 snowballstemmer-1.2.1 sphinx-1.7.9 sphinxcontrib-websupport-1.1.0\n'}
18-09-06:11:29:35 INFO     [clipper_admin.py:456] {'stream': "\x1bYou are using pip version 9.0.3, however version 18.0 is available.\nYou should consider upgrading via the 'pip install --upgrade pip' command.\n\x1b"}
18-09-06:11:29:35 INFO     [clipper_admin.py:456] {'stream': ' ---> 469650387ff8\n'}
18-09-06:11:29:35 INFO     [clipper_admin.py:456] {'aux': {'ID': 'sha256:469650387ff8e4a821d66ae080952ed81a94f680da981f3a3d36dd51acf6c30c'}}
18-09-06:11:29:35 INFO     [clipper_admin.py:456]

You may notice that it seems like we only deployed the predict function to Clipper. However, Clipper will track the dependencies that the model in the predict function uses and include them in the model Docker container as dependencies. You can take a look at the logs above to see the Docker container downloading said dependencies.

Go to http://localhost:3000/pong/ , click 1, and notice how the game is now serving the updated model!

In [27]:
server_proc.terminate()

Cool points here:
1. We never restarted the pong server, but our code seamlessly switches to serving the updated model

## 4. Conclusion

Just as a recap, here's what we did today:

1. Deployed an initial random policy to Clipper and served predictions
2. Trained a new model with data that you generated while playing pong
3. Deployed new version of model and seamlessly switch the model being served

By doing so, we've explored the following Clipper features:

1. Deploy models trained in your choice of framework to Clipper with a few lines of code by using an existing model container or writing your own
2. Easily update or roll back models in running applications
3. Run each model in a separate Docker container for simple cluster management and resource allocation


In [11]:
!python stop_c.py


18-09-05:22:13:12 INFO     [clipper_admin.py:1258] Stopped all Clipper cluster and all model containers
